## `sm_taylor_diagram_mohc_hadgem3-gc31-ll-ssp85.ipynb`: set up HadGEM3-GC3.1-LL so that it can be run with the CanESM5 taylor diagram to compare the output from the future scenario SSP5-85
### NOTE: run`sm_taylor_diagram_cccma-ssp85.ipynb` can be run afterwards to compare one ensemble member from the HadGEM3-GC3.1-LL with Canadian model (SSP5-85 scenario)

In [1]:
import context
import warnings
import intake
import xarray as xr 
import matplotlib.pyplot as plt 
import pandas as pd
import cftime
import gcsfs
import cartopy.crs as ccrs
from pathlib import Path
import pandas as pd
from a448_lib import data_read
import fsspec
import cmocean as cm
import cartopy.feature as cfeature
import numpy as np
import warnings
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
import skill_metrics as sm
from sys import version_info
from skill_metrics import error_check_stats
from skill_metrics import centered_rms_dev

found this_dir=PosixPath('/home/jovyan/work/data_share')
found lib_dir=PosixPath('/home/jovyan/work/data_share/a448_lib')
******************************
context imported. Front of path:
/home/jovyan/work/data_share
back of path: /home/jovyan/.ipython
******************************



### Grab json file with all of the data from cmip6

In [2]:
csv_filename = "pangeo-cmip6.csv"
root = "https://storage.googleapis.com/cmip6"
if Path(csv_filename).is_file():
    print(f"found {csv_filename}")
else:
    print(f"downloading {csv_filename}")
    data_read.download(csv_filename,root=root)
    
json_filename="https://storage.googleapis.com/cmip6/pangeo-cmip6.json"

found pangeo-cmip6.csv


* make a dataframe from the csv version

In [3]:
catalog_df=pd.read_csv(csv_filename)
catalog_df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,hurs,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,tauv,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ta,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,tasmax,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [4]:
col = intake.open_esm_datastore(json_filename)

## Grab all Hadgem-gc31-LL experimental runs

In [5]:
source = "HadGEM3-GC31-LL"
query = dict(
    experiment_id=['ssp585'],
    institution_id = "MOHC",
    source_id = source,
    table_id=["SImon"],
    variable_id=['sithick'])

col_subset = col.search(require_all_on=["source_id"],**query)

In [6]:
list_of_members = col_subset.df

#### Define Taylor Statistics Function

In [7]:
def taylor_statistics(predicted,reference,field=''):
    '''
    Calculates the statistics needed to create a Taylor diagram as 
    described in Taylor (2001) using the data provided in the predicted 
    field (PREDICTED) and the reference field (REFERENCE).
    
    The statistics are returned in the STATS dictionary.
    If a dictionary is provided for PREDICTED or REFERENCE, then 
    the name of the field must be supplied in FIELD.
  
    The function currently supports dictionaries, lists, and np.ndarray,
    types for the PREDICTED and REFERENCE variables.
 
    Input:
    PREDICTED : predicted field
    REFERENCE : reference field
    FIELD     : name of field to use in PREDICTED and REFERENCE dictionaries
                (optional)
    NORM      : logical flag specifying statistics are to be normalized 
                with respect to standard deviation of reference field
                = True,  statistics are normalized
                = False, statistics are not normalized
 
    Output:
    STATS          : dictionary containing statistics
    STATS['ccoef'] : correlation coefficients (R)
    STATS['crmsd'] : centered root-mean-square (RMS) differences (E')
    STATS['sdev']  : standard deviations
 
    Each of these outputs are one-dimensional with the same length. 
    First index corresponds to the reference series for the diagram. 
    For example SDEV[1] is the standard deviation of the reference 
    series (sigma_r) and SDEV[2:N] are the standard deviations of the 
    other (predicted) series.
 
    Reference:
    
    Taylor, K. E. (2001), Summarizing multiple aspects of model 
      performance in a single diagram, J. Geophys. Res., 106(D7),
      7183-7192, doi:10.1029/2000JD900719.
    Author: Peter A. Rochford
        Symplectic, LLC
        www.thesymplectic.com
        prochford@thesymplectic.com
    Created on Dec 3, 2016
    '''

    p, r = error_check_stats(predicted,reference,field)

    # Calculate correlation coefficient
    ccoef = np.corrcoef(p,r)
    ccoef = ccoef[0]

    # Calculate centered root-mean-square (RMS) difference (E')^2
    crmsd = [0.0, centered_rms_dev(p,r)]

    # Calculate standard deviation of predicted field w.r.t N (sigma_p)
    sdevp = np.std(p)
    
    # Calculate standard deviation of reference field w.r.t N (sigma_r)
    sdevr = np.std(r)
    sdev = [sdevr, sdevp];

    # Store statistics in a dictionary
    stats = {'ccoef': ccoef, 'crmsd': crmsd, 'sdev': sdev}
    return stats

#### These are the models we will be comparing. Treat member 0 (r1i1p2f1 as the reference model)

In [8]:
member_ids = list_of_members['member_id']
subset_member_ids = member_ids[0:4]
print(subset_member_ids)

0    r1i1p1f3
1    r3i1p1f3
2    r2i1p1f3
3    r4i1p1f3
Name: member_id, dtype: object


In [ ]:
# setting up reference 
my_memberid = "member_id==" + "\'" + str(subset_member_ids[0]) +"\'"
filenames = list_of_members.query(my_memberid)['zstore'].iloc[0]
reference =xr.open_zarr(fsspec.get_mapper(filenames), consolidated=True)

mohc_ssp585_pd = reference['sithick'][200,:,:].to_dataframe()
mohc_ssp585_pd = mohc_ssp585_pd.dropna()
mohc_ssp585_pd
%store mohc_ssp585_pd